In [9]:
import pandas as pd
import numpy as np 
import os
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

def only_the_words_in_index( list, index ):
    output = []
    for a, s in enumerate(list):
        if s in index:
            output.append(list[a])
    return output

def string_stemmer(line):
    import re
    from nltk.corpus import stopwords
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer("english")
    line = re.sub(r'[^a-zA-Z ]',r'',line)
    line = line.split()
    line = [word for word in line if word not in stopwords.words('english')]  # remove the stop words. 
    output = []
    for word in line:
        output.append(stemmer.stem(word))     #stem all words 
    output = ' '.join(output)           # join the list to make a string
    return output


def get_parameter_for_w2v_for_questions( question_file_path, wiki_folder_path, sizenum, minnum  ):
    sentences = MySentences(wiki_folder_path)
    result = np.zeros(sizenum*minnum).reshape(sizenum,minnum)

    for k in xrange(sizenum):
        for j in xrange(minnum):
            model = gensim.models.Word2Vec(sentences,size=300 + 4*k, window=5, min_count= 1 + j, workers=4)
            myanswer_list = []
            myanswer_distance = pd.DataFrame(np.zeros(4).reshape(1,4), columns = ['A','B','C','D'])    #initialize dataframe to store my answers distance
            train = pd.read_table(question_file_path,sep = '\t')
            for i in train.index.values:
                length = len(train)
                q = train.ix[i][1]
                a1 = train.ix[i][3]
                a2 = train.ix[i][4]
                a3 = train.ix[i][5]
                a4 = train.ix[i][6]

                q = string_stemmer(q)
                a1 = string_stemmer(a1)
                a2 = string_stemmer(a2)
                a3 = string_stemmer(a3)
                a4 = string_stemmer(a4)

                q  = q.split()
                a1 = a1.split()
                a2 = a2.split()
                a3 = a3.split()
                a4 = a4.split()

                q =  only_the_words_in_index( q, model.index2word)
                a1 = only_the_words_in_index( a1, model.index2word)
                a2 = only_the_words_in_index( a2, model.index2word)
                a3 = only_the_words_in_index( a3, model.index2word)
                a4 = only_the_words_in_index( a4, model.index2word)

                try:
                    answer_similarity = np.array([['A',model.n_similarity(q, a1)],['B',model.n_similarity(q, a2)],
                                                  ['C',model.n_similarity(q, a3)],['D',model.n_similarity(q, a4)]])
                except:
                    print 'Error on ', i, ' and set lengths at random'
                    answer_similarity = np.array([['A',np.random.rand()],['B',np.random.rand()],
                                                  ['C',np.random.rand()],['D',np.random.rand()]])

                    myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
                    myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
                    myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
                    myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )

                    #myanswer_index = answer_similarity[:,1].argsort()[-2]        #get the second best 
                    myanswer_index = answer_similarity[:,1].argmax()         #get the maximum similarity 
                    myanswer = answer_similarity[myanswer_index][0]
                    myanswer_list.append(myanswer)
                    continue

                myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
                myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
                myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
                myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )

                #myanswer_index = answer_similarity[:,1].argsort()[-2]
                myanswer_index = answer_similarity[:,1].argmax()          #get the maximum similarity 
                myanswer = answer_similarity[myanswer_index][0]
                myanswer_list.append(myanswer)

                print 'progress: ','k =',k, 'j = ',j ,' : ',i , '/', length 
                #for printing out the distance
                # myanswer_distance.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_distance.csv', encoding='utf-8')    
            train['ck_12_word2vec_answer'] = myanswer_list
            train['ck_12_word2vec_correct'] = (train['correctAnswer'] == train['ck_12_word2vec_answer'])
            result[k,j] = train['ck_12_word2vec_correct'].sum(axis =0) / (len(train) + 0.0)
            print result
    print 'max at: ', result.argmax(axis = 0).max()+1 , ",", result.argmax(axis = 1).max()+1
    return result




def run_w2v_for_questions( question_file_path, wiki_folder_path ):
    sentences = MySentences(wiki_folder_path)

    model = gensim.models.Word2Vec(sentences,size=300 , window=5, min_count= 1 , workers=4)
    myanswer_list = []
    myanswer_distance = pd.DataFrame(np.zeros(4).reshape(1,4), columns = ['A','B','C','D'])    #initialize dataframe to store my answers distance
    train = pd.read_table(question_file_path,sep = '\t')
    for i in train.index.values:
        length = len(train)
        q = train.ix[i][1]
        a1 = train.ix[i][2]
        a2 = train.ix[i][3]
        a3 = train.ix[i][4]
        a4 = train.ix[i][5]

        q = string_stemmer(q)
        a1 = string_stemmer(a1)
        a2 = string_stemmer(a2)
        a3 = string_stemmer(a3)
        a4 = string_stemmer(a4)

        q  = q.split()
        a1 = a1.split()
        a2 = a2.split()
        a3 = a3.split()
        a4 = a4.split()

        q =  only_the_words_in_index( q, model.index2word)
        a1 = only_the_words_in_index( a1, model.index2word)
        a2 = only_the_words_in_index( a2, model.index2word)
        a3 = only_the_words_in_index( a3, model.index2word)
        a4 = only_the_words_in_index( a4, model.index2word)

        try:
            answer_similarity = np.array([['A',model.n_similarity(q, a1)],['B',model.n_similarity(q, a2)],
                                          ['C',model.n_similarity(q, a3)],['D',model.n_similarity(q, a4)]])
        except:
            print 'Error on ', i, ' and set lengths at random'
            answer_similarity = np.array([['A',np.random.rand()],['B',np.random.rand()],
                                          ['C',np.random.rand()],['D',np.random.rand()]])

            myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
            myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
            myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
            myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )

            #myanswer_index = answer_similarity[:,1].argsort()[-2]        #get the second best 
            myanswer_index = answer_similarity[:,1].argmax()         #get the maximum similarity 
            myanswer = answer_similarity[myanswer_index][0]
            myanswer_list.append(myanswer)
            continue

        myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
        myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
        myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
        myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )

        #myanswer_index = answer_similarity[:,1].argsort()[-2]
        myanswer_index = answer_similarity[:,1].argmax()          #get the maximum similarity 
        myanswer = answer_similarity[myanswer_index][0]
        myanswer_list.append(myanswer)

        print 'progress: ',i , '/', length 
        #for printing out the distance
                # myanswer_distance.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_distance.csv', encoding='utf-8')    
    train['correctAnswer'] = myanswer_list
    return train

In [ ]:
question_file_path = '/Users/MK/GitHub/the_answer_is/data/training_set.tsv'
wiki_folder_path = '/Users/MK/GitHub/the_answer_is/data/wikipedia_from_all_ck_words_stemmed'
sizenum = 2
minnum = 2
result = get_parameter_for_w2v_for_questions( question_file_path, wiki_folder_path, sizenum, minnum  )

In [ ]:
question_file_path = '/Users/MK/GitHub/the_answer_is/data/training_set.tsv'
wiki_folder_path = '/Users/MK/GitHub/the_answer_is/data/wikipedia_stemmed_all_merged'
sizenum = 5
minnum = 5
result = get_parameter_for_w2v_for_questions( question_file_path, wiki_folder_path, sizenum, minnum  )

 k = 4 j =  0  :  1022 / 2500
progress:  k = 4 j =  0  :  1023 / 2500
progress:  k = 4 j =  0  :  1024 / 2500
progress:  k = 4 j =  0  :  1025 / 2500
progress:  k = 4 j =  0  :  1026 / 2500
progress:  k = 4 j =  0  :  1027 / 2500
progress:  k = 4 j =  0  :  1028 / 2500
progress:  k = 4 j =  0  :  1029 / 2500
progress:  k = 4 j =  0  :  1030 / 2500
progress:  k = 4 j =  0  :  1031 / 2500
progress:  k = 4 j =  0  :  1032 / 2500
progress:  k = 4 j =  0  :  1033 / 2500
progress:  k = 4 j =  0  :  1034 / 2500
progress:  k = 4 j =  0  :  1035 / 2500
progress:  k = 4 j =  0  :  1036 / 2500
progress:  k = 4 j =  0  :  1037 / 2500
progress:  k = 4 j =  0  :  1038 / 2500
progress:  k = 4 j =  0  :  1039 / 2500
progress:  k = 4 j =  0  :  1040 / 2500
progress:  k = 4 j =  0  :  1041 / 2500
progress:  k = 4 j =  0  :  1042 / 2500
progress:  k = 4 j =  0  :  1043 / 2500
progress:  k = 4 j =  0  :  1044 / 2500
progress:  k = 4 j =  0  :  1045 / 2500
progress:  k = 4 j =  0  :  1046 / 2500
progress: 

In [10]:
question_file_path = '/Users/MK/GitHub/the_answer_is/data/validation_set.tsv'
wiki_folder_path = '/Users/MK/GitHub/the_answer_is/data/wikipedia_stemmed_all_merged'
validation = pd.read_table(question_file_path,sep = '\t')
validation.head()
answer = run_w2v_for_questions( question_file_path, wiki_folder_path  )

progress:  0 / 8132
progress:  1 / 8132
progress:  2 / 8132
progress:  3 / 8132
progress:  4 / 8132
progress: 

/Users/MK/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/MK/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


 5 / 8132
progress:  6 / 8132
Error on  7  and set lengths at random
progress:  8 / 8132
progress:  9 / 8132
progress:  10 / 8132
progress:  11 / 8132
progress:  12 / 8132
progress:  13 / 8132
progress:  14 / 8132
progress:  15 / 8132
progress:  16 / 8132
progress:  17 / 8132
progress:  18 / 8132
progress:  19 / 8132
progress:  20 / 8132
progress:  21 / 8132
progress:  22 / 8132
progress:  23 / 8132
progress:  24 / 8132
progress:  25 / 8132
progress:  26 / 8132
progress:  27 / 8132
progress:  28 / 8132
progress:  29 / 8132
progress:  30 / 8132
progress:  31 / 8132
Error on  32  and set lengths at random
progress:  33 / 8132
Error on  34  and set lengths at random
progress:  35 / 8132
progress:  36 / 8132
progress:  37 / 8132
progress:  38 / 8132
progress:  39 / 8132
progress:  40 / 8132
progress:  41 / 8132
progress:  42 / 8132
progress:  43 / 8132
progress:  44 / 8132
progress:  45 / 8132
progress:  46 / 8132
progress:  47 / 8132
progress:  48 / 8132
progress:  49 / 8132
progress:  50

In [13]:
answer = answer[['id','correctAnswer']]

In [15]:
answer.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/20151210_submission.csv')

In [18]:
answer

,id,correctAnswer
0,102501,A
1,102502,B
2,102503,B
3,102504,D
4,102505,A
5,102506,D
6,102507,B
7,102508,C
8,102509,B
9,102510,C
